In [2]:
import json
import random
import pickle
import numpy as np
import nltk
from re import findall
from nltk.stem import WordNetLemmatizer
import datetime as dt
from tensorflow.keras.models import load_model

In [3]:
lemmantizer = WordNetLemmatizer()

In [4]:
intents = json.loads(open('C:/Users/Admin/Python LL Docs/AI Chatbot/intents.json').read())

In [5]:
words = pickle.load(open("words.pkl", "rb"))

In [6]:
classes = pickle.load(open("classes.pkl", "rb"))

In [7]:
model = load_model("chatbot_model.h5")

In [8]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmantizer.lemmatize(word) for word in sentence_words]
    return sentence_words

In [9]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i]=1
    return np.array(bag)

In [10]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.5
    results = [[i,r] for i,r in enumerate(res) if r > ERROR_THRESHOLD]
    results.sort(key = lambda x: x[1], reverse = True)
    return_list = []
    for r in results:
        return_list.append({"intent":classes[r[0]],"probability": str(r[1])})
    return return_list

In [11]:
def get_response(intents_list,intents_json):
    tag = intents_list[0]["intent"]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

In [12]:
new_vocab = []
time_chat = []

In [15]:
print("Hi, I'm Puzzle, asks me anything?")

while True:
    message = input("")
    message = message.lower()
    
    #temporary
    curr_time = dt.datetime.now()
    time_chat.append({str(curr_time),message})
    #to add vocabulary
    if message not in new_vocab:
        new_vocab.append(message)    
    
    #to break code
    if message.lower() == "exit":
        break
    
    #JSON based chat
    try:
        ints = predict_class(message)
        res = get_response(ints,intents)
        print(res)
        
    except:
        print("Please reframe your sentence and try asking me again.")

Hi, I'm Puzzle, asks me anything?
Exit


In [ ]:
new_vocab

In [ ]:
time_chat